## Import Libraries

In [ ]:
!pip install -U segmentation-models

In [ ]:
!pip install keras==2.2.4

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
from kaggle_datasets import KaggleDatasets
import tensorflow as tf

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.layers import Dropout, Flatten
from tensorflow.keras.layers import Convolution2D,Activation,GlobalAveragePooling2D,MaxPooling2D,Flatten,Dense,Dropout,Input,Reshape,Lambda
from keras.layers.convolutional import Conv2D, MaxPooling2D
import cv2
from sklearn.model_selection import train_test_split
import pickle
import os
import pandas as pd
import random
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau
from keras.models import Model
import math
from keras.callbacks import TensorBoard
import tensorflow.keras.layers as L

import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = [16, 8]

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import segmentation_models as sm
from segmentation_models import PSPNet,Unet
from segmentation_models.metrics import IOUScore, Precision,Recall
from segmentation_models.losses import JaccardLoss,CategoricalFocalLoss,DiceLoss,CategoricalCELoss
from tensorflow.keras.optimizers import RMSprop

print('Using Segmentation version:', sm.__version__)
print('Using Tensorflow version:', tf.__version__)

In [ ]:
sm.framework() 

In [ ]:
sm.set_framework('tf.keras')

In [ ]:
sm.framework() 

In our setup, we:
- created a data/ folder
- created CDS folder which contains all folders
- created 4 folders train_frames train_masks val_frames val_masks
- created train and val subfolders inside 4 folders
- put the pictures in data/CDS/train_frames/train/
- put the label pictures in data/train_masks/train/
In summary, this is our directory structure:
```
    data/
        CDS/
            label_colors.txt
            train_frames/
                train/
                    frame1.png # .png format
            train_masks/
                train/
                    frame1_L.png # _L.png format
            val_frames/
                val/
                    frame2.png # .png format
            val_masks/
                val/
                    frame2_L.png # _L.png format
```

In [ ]:
def parse_code(l):
    '''Function to parse lines in a text file, returns separated elements (label codes and names in this case)
    '''
    if len(l.strip().split("\t")) == 2:
        a, b = l.strip().split("\t")
        return tuple(int(i) for i in a.split(' ')), b
    else:
        a, b, c = l.strip().split("\t")
        return tuple(int(i) for i in a.split(' ')), c
    
def rgb_to_onehot(rgb_image, colormap):
    '''Function to one hot encode RGB mask labels
        Inputs: 
            rgb_image - image matrix (eg. 144 x 144 x 3 dimension numpy ndarray)
            colormap - dictionary of color to label id
        Output: One hot encoded image of dimensions (height x width x num_classes) where num_classes = len(colormap)
    '''
    num_classes = len(colormap)
    shape = rgb_image.shape[:2]+(num_classes,)
    encoded_image = np.zeros( shape, dtype=np.int32 )
    for i, cls in enumerate(colormap):
        encoded_image[:,:,i] = np.all(rgb_image.reshape( (-1,3) ) == colormap[i], axis=1).reshape(shape[:2])
    #print(encoded_image.shape)
    return encoded_image

def onehot_to_rgb(onehot, colormap):
    '''Function to decode encoded mask labels
        Inputs: 
            onehot - one hot encoded image matrix (height x width x num_classes)
            colormap - dictionary of color to label id
        Output: Decoded RGB image (height x width x 3) 
    '''
    single_layer = np.argmax(onehot, axis=-1)
    output = np.zeros( onehot.shape[:2]+(3,) )
    for k in colormap.keys():
        output[single_layer==k] = colormap[k]
    return np.uint8(output)

In [ ]:
IMAGE_SIZE = (384,384)
def TrainAugmentGenerator(seed = 909, batch_size = 16):
    '''Train Image data generator
        Inputs: 
            seed - seed provided to the flow_from_directory function to ensure aligned data flow
            batch_size - number of images to import at a time
        Output: Decoded RGB image (height x width x 3) 
    '''
    train_image_generator = train_frames_datagen.flow_from_directory(
    DATA_PATH + 'train_frames/',
    batch_size = batch_size, seed = seed,target_size=IMAGE_SIZE,shuffle=True,interpolation='nearest')

    train_mask_generator = train_masks_datagen.flow_from_directory(
    DATA_PATH + 'train_masks/',
    batch_size = batch_size, seed = seed,target_size=IMAGE_SIZE,shuffle=True,interpolation='nearest')

    while True:
        X1i = train_image_generator.next()
        X2i = train_mask_generator.next()
        
        # Add new for TPU
#         for j in range(X1i[0].shape[0]):
#             X1i[0][j,:,:,:] = tf.cast(X1i[0][j,:,:,:], tf.float32)/255.0
            #print(X1i[0][j,:,:,:])
        #One hot encoding RGB images
        mask_encoded = [rgb_to_onehot(X2i[0][x,:,:,:], id2code) for x in range(X2i[0].shape[0])]
        #print(mask_encoded.shape)
        yield X1i[0], np.asarray(mask_encoded)

def ValAugmentGenerator(seed = 909, batch_size = 16):
    '''Validation Image data generator
        Inputs: 
            seed - seed provided to the flow_from_directory function to ensure aligned data flow
            batch_size - number of images to import at a time
        Output: Decoded RGB image (height x width x 3) 
    '''
    val_image_generator = val_frames_datagen.flow_from_directory(
    DATA_PATH + 'val_frames/',
    batch_size = batch_size, seed = seed,target_size=IMAGE_SIZE,interpolation='nearest',shuffle=False)


    val_mask_generator = val_masks_datagen.flow_from_directory(
    DATA_PATH + 'val_masks/',
    batch_size = batch_size, seed = seed,target_size=IMAGE_SIZE,interpolation='nearest',shuffle=False)


    while True:
        X1i = val_image_generator.next()
        X2i = val_mask_generator.next()
        
        # Add new for TPU
#         for j in range(X1i[0].shape[0]):
#             X1i[0][j,:,:,:] = tf.cast(X1i[0][j,:,:,:], tf.float32)/255.0
            #print(X1i[0][j,:,:,:])
        #print(X1i[0])
        #One hot encoding RGB images
        mask_encoded = [rgb_to_onehot(X2i[0][x,:,:,:], id2code) for x in range(X2i[0].shape[0])]
        
        yield X1i[0], np.asarray(mask_encoded)

Defind variables

In [ ]:
img_dir = '../input/goodgame-data/GGDataSet/'

DATA_PATH = '../input/goodgame-data/GGDataSet/'

# Seed defined for aligning images and their masks
train_numbers = len(os.listdir(img_dir + 'train_frames/train/'))

val_numbers = len(os.listdir(img_dir + 'val_frames/val/'))

print('Number of train datasets: ',train_numbers)

print('Number of val datasets: ', val_numbers)

BATCH_SIZE = 64

EPOCHS = 10

label_codes, label_names = zip(*[parse_code(l) for l in open(img_dir+"label_colors.txt")])
label_codes, label_names = list(label_codes), list(label_names)

print(label_codes, label_names)

### Create useful label and code conversion dictionaries
These will be used for:
One hot encoding the mask labels for model training
Decoding the predicted labels for interpretation and visualization


In [ ]:
code2id = {v:k for k,v in enumerate(label_codes)}
id2code = {k:v for k,v in enumerate(label_codes)}

name2id = {v:k for k,v in enumerate(label_names)}
id2name = {k:v for k,v in enumerate(label_names)}

print(id2code,id2name)

### Data Aug

In [ ]:
data_gen_args = dict(rotation_range=0.2,
                      width_shift_range=0.05,
                      height_shift_range=0.05,
                      shear_range=0.05,
                      zoom_range=0.05,
                      vertical_flip=True, #can than voi bien bao
                      horizontal_flip=True,
                      fill_mode='nearest',rescale=1./255) # If we use TPU we will use other method to resacle

mask_gen_args = dict(rotation_range=0.2,
                      width_shift_range=0.05,
                      height_shift_range=0.05,
                      shear_range=0.05,
                      zoom_range=0.05,
                      vertical_flip=True, #can than voi bien bao
                      horizontal_flip=True,
                      fill_mode='nearest')

data_gen_args_val = dict(rescale=1./255) # If we use TPU we will use other method to resacle
mask_gen_args_val = dict()

train_frames_datagen = ImageDataGenerator(**data_gen_args)
train_masks_datagen = ImageDataGenerator(**mask_gen_args)
val_frames_datagen = ImageDataGenerator(**data_gen_args_val)
val_masks_datagen = ImageDataGenerator(**mask_gen_args_val)

### Calculate Weights

#### Method 1
When resize to (144,144) or (384,384) we need to calculate weights satify to image size

In [ ]:
import cv2
import os
import numba

@numba.njit(fastmath=True)
def matrix(total_road,total_line,total_background,total_sign,image):
    for j in range(0,image.shape[0]):
        for k in range(0,image.shape[1]):
            b,g,r = image[j][k]
            if (r,g,b) == (128,0,0):
                total_road += 1
            if (r,g,b) == (192,128,128):
                total_line += 1
            if (r,g,b) == (0,0,0):
                total_background +=1
            if (r,g,b) == (102,102,102):
                total_sign += 1
    return total_road,total_line,total_background,total_sign

def calc_weights(masks_folder):
    """Calculate class weights according to classes distribution in a dataset"""
    images_list = os.listdir(masks_folder)
    total_road = 0
    total_line = 0
    total_background = 0
    total_sign = 0
    for i in range(len(images_list)):
        print("Processing ---------",round(i/len(images_list)*100,2),'%\t',i,"/",len(images_list))
        image = cv2.imread(masks_folder + '/' + images_list[i])
        # Resize image to (144,144) or (384,384)
        image = cv2.resize(image,IMAGE_SIZE,interpolation=cv2.INTER_NEAREST) # Noi suy 
        total_road,total_line,total_background,total_sign = matrix(total_road,total_line,total_background,total_sign,image)
        #b,g,r = image[373][145]
        #print(b,g,r)

    total = len(images_list) * image.shape[0] * image.shape[1]
    print("------------------------------------")
    print("Weight of Road: ", total_road / total)
    print("Weight of Line: ", total_line / total)
    print("Weight of BG: ", total_background / total)
    print("Weight of Sign: ", total_sign / total)

IMAGE_SIZE = (144,144)
img_dir = '../input/goodgame-data/GGDataSet/train_masks/train/'

calc_weights(img_dir)

#### Method 2

In [ ]:
import cv2
import os
import numba

@numba.njit(fastmath=True)
def matrix(total_road,total_line,total_background,total_sign,image):
    for j in range(0,480):
        for k in range(0,640):
            b,g,r = image[j][k]
            if (r,g,b) == (128,0,0):
                total_road += 1
            if (r,g,b) == (192,128,128):
                total_line += 1
            if (r,g,b) == (0,0,0):
                total_background +=1
            if (r,g,b) == (102,102,102):
                total_sign += 1
    return total_road,total_line,total_background,total_sign

@numba.njit(fastmath=True)
def cal_contains_object(r_c,g_c,b_c,image):
    for j in range(0,480):
        for k in range(0,640):
            b,g,r = image[j][k]
            if (r,g,b) == (r_c,g_c,b_c):
                return 1
    return 0
def calc_weights(masks_folder):
    """Calculate class weights according to classes distribution in a dataset"""
    global road_global
    global line_global
    global bg_global
    global sign_global    
    
    images_list = os.listdir(masks_folder)
    total_road = 0
    total_line = 0
    total_background = 0
    total_sign = 0
    for i in range(len(images_list)):
        print("Processing ---------",round(i/len(images_list)*100,2),'%\t',i,"/",len(images_list))
        image = cv2.imread(masks_folder + '/' + images_list[i])
        total_road,total_line,total_background,total_sign = matrix(total_road,total_line,total_background,total_sign,image)
        #b,g,r = image[373][145]
        #print(b,g,r)
        road_global += cal_contains_object(128,0,0,image)
        line_global += cal_contains_object(192,128,128,image)
        bg_global += cal_contains_object(0,0,0,image)
        sign_global += cal_contains_object(102,102,102,image)
        
    print("Total of road images: ", road_global)
    print("Total of line images: ", line_global)
    print("Total of background images: ", bg_global)
    print("Total of sign images: ", sign_global)

    road_global = road_global * 640 * 480
    line_global = line_global * 640 * 480
    bg_global = bg_global * 640 * 480
    sign_global = sign_global * 640 * 480

    print("------------------------------------")
    print("Weight of Road: ", total_road / road_global)
    print("Weight of Line: ", total_line / line_global)
    print("Weight of BG: ", total_background / bg_global)
    print("Weight of Sign: ", total_sign / sign_global)

road_global = 0
line_global = 0
bg_global = 0
sign_global = 0

img_dir = '../input/goodgame-data/GGDataSet/train_frames/train'

calc_weights(img_dir)

### Cusom loss

In [ ]:
import tensorflow.keras.backend as K

def class_tversky(y_true, y_pred):
    smooth = 1

    y_true = K.permute_dimensions(y_true, (3,1,2,0))
    y_pred = K.permute_dimensions(y_pred, (3,1,2,0))

    y_true_pos = K.batch_flatten(y_true)
    y_pred_pos = K.batch_flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos, 1)
    false_neg = K.sum(y_true_pos * (1-y_pred_pos), 1)
    false_pos = K.sum((1-y_true_pos)*y_pred_pos, 1)
    alpha = 0.7
    return (true_pos + smooth)/(true_pos + alpha*false_neg + (1-alpha)*false_pos + smooth)

def focal_tversky_loss(y_true,y_pred):
    pt_1 = class_tversky(y_true, y_pred)
    gamma = 0.75
    return K.sum(K.pow((1-pt_1), gamma))

def categorical_focal_loss_fixed(y_true, y_pred):
    """
    :param y_true: A tensor of the same shape as `y_pred`
    :param y_pred: A tensor resulting from a softmax
    :return: Output tensor.
    """
    alpha = .25
    gamma = 2.

    # Scale predictions so that the class probas of each sample sum to 1
    y_pred /= K.sum(y_pred, axis=-1, keepdims=True)

    # Clip the prediction value to prevent NaN's and Inf's
    epsilon = K.epsilon()
    
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    
    # Calculate Cross Entropy
    cross_entropy = -y_true * K.log(y_pred)

    # Calculate Focal Loss
    loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy

    # Compute mean loss in mini_batch
    return K.mean(K.sum(loss, axis=-1))

def combine_loss(y_true,y_pred):
    tversky_loss = focal_tversky_loss(y_true,y_pred)
    categorical_focal_loss = categorical_focal_loss_fixed(y_true,y_pred)
    loss = tversky_loss + categorical_focal_loss
    return loss

### Load Model

In [ ]:
IMG_SIZE = (384,384,3)
model = PSPNet(backbone_name='efficientnetb4',
                   input_shape=IMG_SIZE,
                   classes=4,
                   activation='softmax',
                   weights= None,
                   encoder_weights='imagenet',
                   encoder_freeze=True,
                   downsample_factor=8,
                   psp_conv_filters=512,
                   psp_pooling_type='avg',
                   psp_use_batchnorm=True,
                   psp_dropout=0.25)

# model = Unet(backbone_name='resnet34', 
#                          input_shape=(384, 384, 3), 
#                          classes=4, 
#                          activation='softmax', 
#                          weights=None, 
#                          encoder_weights='imagenet', 
#                          encoder_freeze=False, 
#                          encoder_features='default', 
#                          decoder_block_type='upsampling',
#                          decoder_filters=(256, 128, 64, 32, 16), 
#                          decoder_use_batchnorm=True)
#Load Pre Model If you had:
#model.load_weights('../input/abcdef/efficientnetb31_epoch-006_loss-0.7743_val_loss-0.7764.h5')

#List labels map 
#0-road, 1-line, 2-background, 3 - Traffic sign

'''
Weight of method-1
------------------------------------
Weight of Road:  0.3645078248104593
Weight of Line:  0.028078782297801714
Weight of BG:  0.5667032257035932
Weight of Sign:  0.040710167188145796
'''

'''
Weight of method-2
------------------------------------
Weight of Road:  0.3645078248104593
Weight of Line:  0.02810491830366377
Weight of BG:  0.5667032257035932
Weight of Sign:  0.06350628595210751
'''

'''
------------------------------------
Weight of Road:  0.34606781118093866
Weight of Line:  0.027605678363214644
Weight of BG:  0.6213731314385866
Weight of Sign:  0.011814638469397927
'''

'''
Weight for image size: (384,384)
------------------------------------
Weight of Road:  0.36446377094056215
Weight of Line:  0.028047958303599876
Weight of BG:  0.5667768587313581
Weight of Sign:  0.04071141202447992
'''

'''
Weight for image size: (480,480)
------------------------------------
Weight of Road:  0.3646129968313012
Weight of Line:  0.02808243137356203
Weight of BG:  0.5665942663665875
Weight of Sign:  0.04071030542854929
'''

'''
Weight for image size: (144,144)
------------------------------------
Weight of Road:  0.36310559985228047
Weight of Line:  0.027933321589439203
Weight of BG:  0.5682527441486227
Weight of Sign:  0.04070833440965764
'''
class_weights_dice = [0.36446377094056215,0.028047958303599876,0.5667768587313581,0.04071141202447992]
class_indexes_dice = [0,1,2,3]
# class_indexs_focal = [3,4,5,6,7,8]

# --------------------------------------------------------------------
#List of the loss value
jaccard_loss = JaccardLoss(class_weights=class_weights_dice, class_indexes=class_indexes_dice,per_image=True)
dice_loss = DiceLoss(beta = 0.5,class_weights=class_weights_dice, class_indexes=class_indexes_dice) #khop anh 

# binary_focal_loss = BinaryFocalLoss()
categorical_focal_loss = CategoricalFocalLoss(alpha=0.25, gamma=2,class_indexes=[1])

# binary_crossentropy = BinaryCELoss()
categorical_crossentropy = CategoricalCELoss(class_weights=class_weights_dice, class_indexes=class_indexes_dice)

# loss combinations
# bce_dice_loss = binary_crossentropy + dice_loss
# bce_jaccard_loss = binary_crossentropy + jaccard_loss

cce_dice_loss = categorical_crossentropy + dice_loss
cce_jaccard_loss = categorical_crossentropy + jaccard_loss

# binary_focal_dice_loss = binary_focal_loss + dice_loss
# binary_focal_jaccard_loss = binary_focal_loss + jaccard_loss

categorical_focal_dice_loss = dice_loss + ( 1 * categorical_focal_loss )  # Dat Vu edited here
categorical_focal_jaccard_loss = categorical_focal_loss + jaccard_loss 
#------------------------------------------------------------------------

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5),Precision(threshold=0.5), Recall(threshold=0.5)]



model.compile(optimizer=RMSprop(lr=0.0001), loss = cce_jaccard_loss, metrics = [metrics])

#tb = TensorBoard(log_dir='logs', write_graph=True)
model.summary()

In [ ]:
BATCH_SIZE = 16
EPOCHS = 10
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(mode='auto', filepath='./efficientnetb4_epoch-{epoch:03d}_loss-{loss:.4f}_val_loss-{val_loss:.4f}.h5', 
                     monitor='val_loss',  
                     save_weights_only='True', 
                     period=1,
                     verbose=1)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                               min_delta=0.0,
                               patience=10,
                               verbose=1)
reduce_learning_rate = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                         factor=0.2,
                                         patience=6,
                                         verbose=1,
                                         epsilon=0.001,
                                         cooldown=0,
                                         min_lr=0.00001)
    
steps_per_epoch = np.ceil(float(train_numbers) / float(BATCH_SIZE))
validation_steps = np.ceil(float(val_numbers) / float(BATCH_SIZE))
result = model.fit_generator(TrainAugmentGenerator(batch_size=BATCH_SIZE), steps_per_epoch=steps_per_epoch,
                validation_data = ValAugmentGenerator(batch_size=BATCH_SIZE), 
                validation_steps = validation_steps, epochs=EPOCHS,callbacks=[model_checkpoint,early_stopping,reduce_learning_rate])

In [ ]:
plt.figure(figsize=(20, 5))

plt.subplot(121)

# Get training and test loss histories
training_loss = result.history['loss']
test_loss = result.history['val_loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.title('Model loss')

plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')

# Pylot iou-score 
plt.subplot(122)

# Get training and test loss histories
training_loss = result.history['iou_score']
test_loss = result.history['val_iou_score']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.title('Model iou_score')
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Score', 'Val IoU Score'])
plt.xlabel('Epoch')
plt.ylabel('IoU Score')

plt.show()






### Unfreeze layers

### Custom loss
## Lovasz Softmax

In [ ]:
"""
Lovasz-Softmax and Jaccard hinge loss in Tensorflow
Maxim Berman 2018 ESAT-PSI KU Leuven (MIT License)
"""

from __future__ import print_function, division

import tensorflow as tf
import numpy as np

def lovasz_grad(gt_sorted):
    """
    Computes gradient of the Lovasz extension w.r.t sorted errors
    See Alg. 1 in paper
    """
    gts = tf.reduce_sum(gt_sorted)
    intersection = gts - tf.cumsum(gt_sorted)
    union = gts + tf.cumsum(1. - gt_sorted)
    jaccard = 1. - intersection / union
    jaccard = tf.concat((jaccard[0:1], jaccard[1:] - jaccard[:-1]), 0)
    return jaccard


# --------------------------- BINARY LOSSES ---------------------------


def lovasz_hinge(logits, labels, per_image=True, ignore=None):
    """
    Binary Lovasz hinge loss
      logits: [B, H, W] Variable, logits at each pixel (between -\infty and +\infty)
      labels: [B, H, W] Tensor, binary ground truth masks (0 or 1)
      per_image: compute the loss per image instead of per batch
      ignore: void class id
    """
    if per_image:
        def treat_image(log_lab):
            log, lab = log_lab
            log, lab = tf.expand_dims(log, 0), tf.expand_dims(lab, 0)
            log, lab = flatten_binary_scores(log, lab, ignore)
            return lovasz_hinge_flat(log, lab)
        losses = tf.map_fn(treat_image, (logits, labels), dtype=tf.float32)
        loss = tf.reduce_mean(losses)
    else:
        loss = lovasz_hinge_flat(*flatten_binary_scores(logits, labels, ignore))
    return loss


def lovasz_hinge_flat(logits, labels):
    """
    Binary Lovasz hinge loss
      logits: [P] Variable, logits at each prediction (between -\infty and +\infty)
      labels: [P] Tensor, binary ground truth labels (0 or 1)
      ignore: label to ignore
    """

    def compute_loss():
        labelsf = tf.cast(labels, logits.dtype)
        signs = 2. * labelsf - 1.
        errors = 1. - logits * tf.stop_gradient(signs)
        errors_sorted, perm = tf.nn.top_k(errors, k=tf.shape(errors)[0], name="descending_sort")
        gt_sorted = tf.gather(labelsf, perm)
        grad = lovasz_grad(gt_sorted)
        loss = tf.tensordot(tf.nn.relu(errors_sorted), tf.stop_gradient(grad), 1, name="loss_non_void")
        return loss

    # deal with the void prediction case (only void pixels)
    loss = tf.cond(tf.equal(tf.shape(logits)[0], 0),
                   lambda: tf.reduce_sum(logits) * 0.,
                   compute_loss,
                   strict=True,
                   name="loss"
                   )
    return loss


def flatten_binary_scores(scores, labels, ignore=None):
    """
    Flattens predictions in the batch (binary case)
    Remove labels equal to 'ignore'
    """
    scores = tf.reshape(scores, (-1,))
    labels = tf.reshape(labels, (-1,))
    if ignore is None:
        return scores, labels
    valid = tf.not_equal(labels, ignore)
    vscores = tf.boolean_mask(scores, valid, name='valid_scores')
    vlabels = tf.boolean_mask(labels, valid, name='valid_labels')
    return vscores, vlabels


# --------------------------- MULTICLASS LOSSES ---------------------------


def lovasz_softmax(probas, labels, classes='present', per_image=False, ignore=None, order='BHWC'):
    """
    Multi-class Lovasz-Softmax loss
      probas: [B, H, W, C] or [B, C, H, W] Variable, class probabilities at each prediction (between 0 and 1)
      labels: [B, H, W] Tensor, ground truth labels (between 0 and C - 1)
      classes: 'all' for all, 'present' for classes present in labels, or a list of classes to average.
      per_image: compute the loss per image instead of per batch
      ignore: void class labels
      order: use BHWC or BCHW
    """
    print(probas,labels)
    if per_image:
        def treat_image(prob_lab):
            prob, lab = prob_lab
            prob, lab = tf.expand_dims(prob, 0), tf.expand_dims(lab, 0)
            prob, lab = flatten_probas(prob, lab, ignore, order)
            return lovasz_softmax_flat(prob, lab, classes=classes)
        losses = tf.map_fn(treat_image, (probas, labels), dtype=tf.float32)
        loss = tf.reduce_mean(losses)
    else:
        loss = lovasz_softmax_flat(*flatten_probas(probas, labels, ignore, order), classes=classes)
    return loss


def lovasz_softmax_flat(probas, labels, classes='present'):
    """
    Multi-class Lovasz-Softmax loss
      probas: [P, C] Variable, class probabilities at each prediction (between 0 and 1)
      labels: [P] Tensor, ground truth labels (between 0 and C - 1)
      classes: 'all' for all, 'present' for classes present in labels, or a list of classes to average.
    """
    C = 1
    losses = []
    present = []
    class_to_sum = list(range(C)) if classes in ['all', 'present'] else classes
    for c in class_to_sum:
        fg = tf.cast(tf.equal(labels, c), probas.dtype)  # foreground for class c
        if classes == 'present':
            present.append(tf.reduce_sum(fg) > 0)
        errors = tf.abs(fg - probas[:, c])
        errors_sorted, perm = tf.nn.top_k(errors, k=tf.shape(errors)[0], name="descending_sort_{}".format(c))
        fg_sorted = tf.gather(fg, perm)
        grad = lovasz_grad(fg_sorted)
        losses.append(
            tf.tensordot(errors_sorted, tf.stop_gradient(grad), 1, name="loss_class_{}".format(c))
                      )
    if len(class_to_sum) == 1:  # short-circuit mean when only one class
        return losses[0]
    losses_tensor = tf.stack(losses)
    if classes == 'present':
        present = tf.stack(present)
        losses_tensor = tf.boolean_mask(losses_tensor, present)
    loss = tf.reduce_mean(losses_tensor)
    return loss


def flatten_probas(probas, labels, ignore=None, order='BHWC'):
    """
    Flattens predictions in the batch
    """
    if order == 'BCHW':
        probas = tf.transpose(probas, (0, 2, 3, 1), name="BCHW_to_BHWC")
        order = 'BHWC'
    if order != 'BHWC':
        raise NotImplementedError('Order {} unknown'.format(order))
    C = 1
    probas = tf.reshape(probas, (-1, C))
    labels = tf.reshape(labels, (-1,))
    if ignore is None:
        return probas, labels
    valid = tf.not_equal(labels, ignore)
    vprobas = tf.boolean_mask(probas, valid, name='valid_probas')
    vlabels = tf.boolean_mask(labels, valid, name='valid_labels')
    return vprobas, vlabels

def keras_lovasz_softmax(labels,probas):
    #return lovasz_softmax(probas, labels)+binary_crossentropy(labels, probas)
    return lovasz_softmax(probas, labels,per_image=True)

In [ ]:
model = PSPNet(backbone_name='efficientnetb4',
                   input_shape=(384, 384, 3),
                   classes=4,
                   activation='softmax',
                   weights= './efficientnetb4_epoch-010_loss-0.7882_val_loss-0.7813.h5',
                   encoder_weights='imagenet',
                   encoder_freeze=False,
                   downsample_factor=8,
                   psp_conv_filters=512,
                   psp_pooling_type='avg')
#                    psp_use_batchnorm=True,
#                    psp_dropout=0.25)
for layer in model.layers:
    layer.trainable = True

model.summary()

In [ ]:
EPOCHS = 20 # Update Epochs
BATCH_SIZE = 16
steps_per_epoch = np.ceil(float(train_numbers) / float(BATCH_SIZE))
validation_steps = np.ceil(float(val_numbers) / float(BATCH_SIZE))
model.compile(optimizer=RMSprop(lr=0.0001), loss = cce_jaccard_loss, metrics = [metrics])

result = model.fit_generator(TrainAugmentGenerator(batch_size=BATCH_SIZE), steps_per_epoch=steps_per_epoch,
                validation_data = ValAugmentGenerator(batch_size=BATCH_SIZE), 
                validation_steps = validation_steps, epochs=EPOCHS,callbacks=[model_checkpoint,early_stopping,reduce_learning_rate])

In [ ]:
plt.figure(figsize=(20, 5))

plt.subplot(121)

# Get training and test loss histories
training_loss = result.history['loss']
test_loss = result.history['val_loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.title('Model loss')

plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')

# Pylot iou-score 
plt.subplot(122)

# Get training and test loss histories
training_loss = result.history['iou_score']
test_loss = result.history['val_iou_score']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.title('Model iou_score')
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Score', 'Val IoU Score'])
plt.xlabel('Epoch')
plt.ylabel('IoU Score')

plt.show()

In [ ]:
# Get training and test loss histories
training_loss = result.history['f1-score']
test_loss = result.history['val_f1-score']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Score', 'Val F1-Score'])
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.show()

In [ ]:
plt.figure(figsize=(20, 5))

plt.subplot(121)

# Get training and test loss histories
training_loss = result.history['precision']
test_loss = result.history['val_precision']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.title('Model Precision')

plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Presicion', 'Test Presicion'])
plt.xlabel('Epoch')
plt.ylabel('Presicion Score')

# Pylot iou-score 
plt.subplot(122)

# Get training and test loss histories
training_loss = result.history['recall']
test_loss = result.history['val_recall']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.title('Model Recall')
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Score', 'Val Recall Score'])
plt.xlabel('Epoch')
plt.ylabel('IoU Score')

plt.show()

### Stage 3: 
Training with Weighted Cross-Entropy + Dice Loss with beta = 1

In [ ]:
dice_loss = DiceLoss(beta = 1,class_weights=class_weights_dice, class_indexes=class_indexes_dice) #khop anh 

categorical_crossentropy = CategoricalCELoss(class_weights=class_weights_dice, class_indexes=class_indexes_dice)

cce_dice_loss = categorical_crossentropy + dice_loss

model = PSPNet(backbone_name='efficientnetb4',
                   input_shape=(384, 384, 3),
                   classes=4,
                   activation='softmax',
                   weights= './efficientnetb4_epoch-020_loss-0.7747_val_loss-0.7731.h5',
                   encoder_weights='imagenet',
                   encoder_freeze=False,
                   downsample_factor=8,
                   psp_conv_filters=512,
                   psp_pooling_type='avg')
#                    psp_use_batchnorm=True,
#                    psp_dropout=0.25)
for layer in model.layers:
    layer.trainable = True

model.summary()

EPOCHS = 30 # Update Epochs
BATCH_SIZE = 16
steps_per_epoch = np.ceil(float(train_numbers) / float(BATCH_SIZE))
validation_steps = np.ceil(float(val_numbers) / float(BATCH_SIZE))
model.compile(optimizer=RMSprop(lr=0.0001), loss = cce_jaccard_loss, metrics = [metrics])

result = model.fit_generator(TrainAugmentGenerator(batch_size=BATCH_SIZE), steps_per_epoch=steps_per_epoch,
                validation_data = ValAugmentGenerator(batch_size=BATCH_SIZE), 
                validation_steps = validation_steps, epochs=EPOCHS,callbacks=[model_checkpoint,early_stopping,reduce_learning_rate])

### Predict Video

In [ ]:
model = PSPNet(backbone_name='efficientnetb4',
                   input_shape=(384, 384, 3),
                   classes=4,
                   activation='softmax',
                   weights= None,
                   encoder_weights='imagenet',
                   encoder_freeze=False,
                   downsample_factor=8,
                   psp_conv_filters=512,
                   psp_pooling_type='avg',
                   psp_use_batchnorm=True,
                   psp_dropout=0.25)

model.load_weights('./efficientnetb4_epoch-020_loss-0.7733_val_loss-0.7726.h5')

In [ ]:
from scipy import interpolate
from scipy.interpolate import RectBivariateSpline

def onehot_to_rgb(onehot, colormap):
    '''Function to decode encoded mask labels
        Inputs: 
            onehot - one hot encoded image matrix (height x width x num_classes)
            colormap - dictionary of color to label id
        Output: Decoded RGB image (height x width x 3) 
    '''
    single_layer = np.argmax(onehot, axis=-1)
    output = np.zeros((384,384,3))
    for k in colormap.keys():
        output[single_layer==k] = colormap[k]
    # Interplotation
#     line = np.zeros((144,144))
#     line[single_layer == 1] = 255    
    
# #     indices = np.array([[0, 0]])
#     x = np.array([],dtype = np.int8)
#     y = np.array([],dtype = np.int8)
#     for i in range(0,144):
#         for j in range(0,144):
#             if line[i][j] == 255:
# #                 arr = np.array([[j,i]])
# #                 indices = np.append(indices,arr,axis=0)
#                 x = np.append(x,[j],axis=0)
#                 y = np.append(y,[i],axis=0)
# #     print(x[0],y[0])
# #     indices = np.delete(indices, 0, axis=0) # Remove first element

#     M, N = line.shape
    
#     spline = RectBivariateSpline(np.arange(M), np.arange(N), line)
#     interpolated = spline(x, y, grid=False)
    
#     print(interpolated)
#     nans = np.zeros(interpolated.shape) + np.nan
#     #print(nans)
#     x_in_bounds = (0 <= x) & (x < M)
#     y_in_bounds = (0 <= y) & (y < N)
#     bounded = np.where(x_in_bounds & y_in_bounds, interpolated, nans)
#     #print(bounded)

    
#     xx, yy = np.meshgrid(x, y)
#     z = np.sin(xx**2+yy**2)
#     f = interpolate.interp2d(x, y, z, kind='cubic')
    
#     for i in range(0,144):
#         for j in range(0,144):
#             if line[i][j] != 255:
#                 znew = f(j, i)
#                 if znew != null:
#                     print(znew)
                
                

#     line = np.zeros((144,144))
#     line[single_layer == 1] = 255
#     line = output[72:,:]
#     for i in range(0,72):
#         for j in range(0,144):
#             if line_opencv[i][j] == 255:
#                 line[i][j] = 255
    return np.uint8(output)#,np.uint8(line)


import cv2
import numpy as np
capture = cv2.VideoCapture('../input/videoo/Video_Beta.avi')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('./output_2007_v4_3.avi',fourcc, 30.0, (384,384))
i=0
while(True):
    ret = capture.grab()
    #if i % 10 == 0:
        
    ret, frame = capture.retrieve()
    
    img = cv2.resize(frame,(384,384),interpolation=cv2.INTER_NEAREST)
    
    lower = np.array([0, 0, 200])
    upper = np.array([180, 255, 255])
    # Create HSV Image and threshold into a range.
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    line_opencv = mask[72:,:] # We want to get half of image below
    
    img = (img[...,::-1].astype(np.float32)) / 255.0
    img = np.reshape(img, (1, 384, 384, 3))
    predict_one = model.predict(img)
    
    #frame_real = cv2.resize(frame,(144,144),interpolation=cv2.INTER_NEAREST)
    frame_pre = onehot_to_rgb(predict_one[0],id2code)
    #cv2.imwrite('test.png',frame_pre)
#     line = cv2.resize(line,(144,144))
    
    #frame_total = np.concatenate((frame_real, frame_pre), axis=0)
    
    i += 1
    print("Frame thu: ",i)
    #GG
    #cv2.imshow("Test",frame_total)
    #Save video
    out.write(frame_pre)
    #cv2.imwrite('Frame_'+ str(i) +'.png',frame_total)
        # do something with frame
capture.release()

### Post Processing

Step 1: Cubic spine interpolation

In [ ]:
from scipy import interpolate
x = np.arange(-5.01, 5.01, 0.25)
y = np.arange(-5.01, 5.01, 0.25)
xx, yy = np.meshgrid(x, y)
z = np.sin(xx**2+yy**2)
f = interpolate.interp2d(x, y, z, kind='cubic')

In [ ]:
xnew = np.arange(-5.01, 5.01, 1e-2)
ynew = np.arange(-5.01, 5.01, 1e-2)
znew = f(xnew, ynew)
plt.plot(x, z[0, :], 'ro-', xnew, znew[0, :], 'b-')
plt.show()

### Convert model to .pb frozen graph 

In [ ]:
from keras import backend as K
# This line must be executed before loading Keras model.
K.set_learning_phase(0)
import tensorflow as tf

def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph
    
model.load_weights('/home/dylan/Desktop/Goodgame_Training/model/Model_4_class_Test_1_epoch-175_loss-0.7673_val_loss-0.7632.h5')

frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])
tf.train.write_graph(frozen_graph, "model_pb", "Model_4_class_Test_1_epoch-175_loss-0.7673_val_loss-0.7632.pb", as_text=False)

print("Input Model:  ",model.inputs)
print("Output Model:  ",model.outputs)

### Read image

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
plt.figure(figsize=(16,10))

img = cv2.imread('../input/preimages/data_beta_bouns(319).png')
img = cv2.resize(img,(144,144))
plt.subplot(1,2,1)
plt.imshow(img)
lower = np.array([0, 0, 220])
upper = np.array([180, 255, 255])
# Create HSV Image and threshold into a range.
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv, lower, upper)
line_opencv = mask[72:,:] # We want to get half of image below
#print(line_opencv.shape)
plt.subplot(1,2,2)
plt.imshow(line_opencv,cmap="gray")

In [ ]:
def onehot_to_rgb(onehot, colormap,line_opencv):
    '''Function to decode encoded mask labels
        Inputs: 
            onehot - one hot encoded image matrix (height x width x num_classes)
            colormap - dictionary of color to label id
        Output: Decoded RGB image (height x width x 3) 
    '''
    single_layer = np.argmax(onehot, axis=-1)
    output = np.zeros((144,144))
#     for k in colormap.keys():
#         output[single_layer==k] = colormap[k]
    output[single_layer == 1] = 255
    output = output[72:,:]
    for i in range(0,72):
        for j in range(0,144):
            if line_opencv[i][j] == 255:
                output[i][j] = 255
    return np.uint8(output)

frame = cv2.imread('../input/preimages/data_beta_bouns(319).png')
img = cv2.resize(frame,(144,144))
img = (img[...,::-1].astype(np.float32)) / 255.0
img = np.reshape(img, (1, 144, 144, 3))
predict_one = model.predict(img)

frame_real = cv2.resize(frame,(144,144))
frame_pre = onehot_to_rgb(predict_one[0],id2code,line_opencv)

cv2.imwrite('./line.png',frame_pre)
#frame_total = np.concatenate((frame_real, frame_pre), axis=1)
plt.imshow(frame_pre,cmap="gray")